### Load trk file, filter on streamline length, cluster, filter on cluster size

In [1]:
def process(ftrk,ffa,fdwi):
    import postprocessing4ExploreDTIdata as p
    from common import save_trk_old_style
    
    fa,affine,data,streamlines = p.importData(ftrk,ffa,fdwi)
    new_streamlines = p.filterLength(streamlines, 2.5)
    clusters = p.computeQuickBundles(new_streamlines,1.)
    new2_streamlines = p.filterSmallBundles(new_streamlines,clusters,10)
    new_clusters = p.computeQuickBundles(new2_streamlines,1.)
    new_lengths = p.getLengths(new2_streamlines)
    fnew_centroidtractogram = dname + 'centroidtractogram.trk'
    save_trk_old_style(fnew_centroidtractogram, new_clusters.centroids, affine, fa.shape)
    fnew_tractogram = dname + 'filteredtractogram.trk'
    save_trk_old_style(fnew_tractogram, new2_streamlines, affine, fa.shape)
    
    return (new2_streamlines,new_clusters,new_lengths)

### Print info on the number of streamlines, bundles and min and max length

In [2]:
def getData4Analyses(streamlines,clusters,lengths):
    print('Nb. streamlines, Nb. Bundles, Min length, Max length')
    print len(streamlines),len(clusters),min(lengths),max(lengths)

### Loop over the previous functions for all the subjects

In [3]:
#subjects = ['P2_F25','P2_F28','P4_F07','P8_F10','P16_F21','P32_F16','P32_F17','P32_F19','P64_F01','P64_F22']
subjects =['P2_F25']#,'P8_F10']

for i in range(len(subjects)):
    dname = '/Volumes/Samsung_T1/dti/ExploreDTI/'+subjects[i]+'/'
    ftrk = dname + '2dseq_conv_MD_C_native_Tracts_DTI.trk'
    ffa = dname + '2dseq_conv_MD_C_native_FA.nii.gz'
    fdwi = dname + '2dseq_conv.nii'
    print subjects[i]
    new2_streamlines,new_clusters,new_lengths = process(ftrk,ffa,fdwi)
    
    f = open(dname+'streamlinesLength.txt', 'w')
    getData4Analyses(new2_streamlines,new_clusters,new_lengths)
    for item in new_lengths:
        f.write("%s\n" % item)

P2_F25
Nb. streamlines, Nb. Bundles, Min length, Max length
13943 324 2.52 27.84


### Get histogram values of the distribution of streamline length

In [4]:
def getHistBins (tab, nbins, mini, maxi):
    stepSize = (maxi-mini)/nbins
    #print stepSize
    print 'minBinVal maxBinVal nbStreamlines'
    for n in range(nbins):
        j = 0
        for i in tab:
            if (i > (mini+n*stepSize)) & (i <(mini+(n+1)*stepSize)):
                j = j+1
        print mini+n*stepSize, mini+(n+1)*stepSize, j
        

In [5]:
getHistBins(new_lengths, 10, 2.5, 75)

minBinVal maxBinVal nbStreamlines
2.5 9.75 12716
9.75 17.0 1082
17.0 24.25 126
24.25 31.5 19
31.5 38.75 0
38.75 46.0 0
46.0 53.25 0
53.25 60.5 0
60.5 67.75 0
67.75 75.0 0


### Import length file

In [8]:
def importLengthFile(dname):
    tab = []

    with open(dname+'streamlinesLength_col.txt', 'rb') as f:
        reader = csv.reader(f)
        tab = list(reader)

    for n in range(len(tab)):
        tab[n] = float(tab[n][0])

    print(subjects[i])
    #print tab[:10]
    tab.sort()
    #print tab[:10]
    return tab

### Get information on the distribution of streamlines in the first and last quantile of the size thr

In [6]:
def lengthQuantiles (tab, thr):
    import numpy as np
    
    first_quantile = max(tab)*thr
    last_quantile = max(tab)*(1-thr)

    n_first = 0
    n_last = 0
    firstQ = []
    lastQ = []
    for t in range(len(tab)):
        if tab[t] < first_quantile:
            n_first=n_first+1
            firstQ.append(tab[t])
        else :    
            if tab[t] > last_quantile:
                n_last=n_last+1
                lastQ.append(tab[t])
    
    median_firstQ = np.median(firstQ)
    median_lastQ = np.median(lastQ)
    print 'min','max','first_quantile','last_quantile','n_first','n_last','median_firstQ','median_lastQ','medianQ_ratio'
    print min(tab),max(tab),first_quantile,last_quantile,n_first,n_last, median_firstQ,median_lastQ, median_lastQ/median_firstQ

In [9]:
import csv

#subjects = ['P2_F25','P2_F28','P4_F07','P8_F10','P16_F21','P32_F16','P32_F17','P32_F19','P64_F01','P64_F22']
subjects =['P2_F25']

for i in range(len(subjects)):
    dname = '/Volumes/Samsung_T1/dti/ExploreDTI/'+subjects[i]+'/'
    
    tab = importLengthFile(dname)
    
    print len(tab)
    
    lengthQuantiles(tab,0.3)
    
    #getHistBins(tab, 10, 2.5, 50)

P2_F25
13943
min max first_quantile last_quantile n_first n_last median_firstQ median_lastQ medianQ_ratio
2.52 27.84 8.352 19.488 11754 86 4.32 21.54 4.98611111111
